In [ ]:
from rich.pretty import pprint

from rage.loaders import PDFLoaeder
from rage.splitters import WordTextSplitter
from rage.retriever import Retriever

In [ ]:
loader = PDFLoaeder()
splitter = WordTextSplitter()
retriever = Retriever()

In [ ]:
file_path = "/resources/pdf/zaratustra.pdf"
documents = await loader.load(source_path=file_path)
pprint(documents[1])

In [ ]:
text_chunks = splitter.split_documents(documents=documents)

print(f"text_chunks => {len(text_chunks)}")
pprint(text_chunks[:5])
pprint(text_chunks[-1])

In [ ]:
print(text_chunks[-1].text)

In [ ]:
collection_name = "zaratustra"
retriever.create_collection(collection_name=collection_name)
retriever.insert_text_chunks(
    collection_name=collection_name,
    text_chunks=text_chunks,
)

In [ ]:
query = "Que quiere el gran Dragón?"
retriever_items = await retriever.hybrid_search(
    collection_name=collection_name,
    query=query,
)

pprint(retriever_items)

In [ ]:
retriever_items = await retriever.dense_search(
    collection_name=collection_name,
    query=query,
)

pprint(retriever_items)

In [ ]:
retriever_items = await retriever.retrieve(
    collection_name=collection_name,
    query=query,
)

pprint(retriever_items)